In [98]:
import sagemaker
import boto3
import os
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session, get_execution_role
from pathlib import Path

In [112]:
import pandas as pd

role = get_execution_role()
bucket="project-4-files"
calc_test_key = "csv/calc_case_description_test_set.csv"
data_location = "s3://{}/{}".format(bucket, calc_test_key)

calc_test_df = pd.read_csv(data_location)
calc_test_df.head()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...


In [113]:
# extract the csv from the csv folder in buacket

s3_client = boto3.client("s3")

os.makedirs("/home/ec2-user/SageMaker/csv", exist_ok=True)

data_locations = [
    "s3://project-4-files/csv/"
]

for data_location in data_locations:
    s3_uri_parts = data_location.split("/")
    bucket = s3_uri_parts[2]

    local_folder_path = "/home/ec2-user/SageMaker/csv"
    
    try:
        for obj in s3_client.list_objects(Bucket=bucket, Prefix="csv/")["Contents"]:
            data_key = obj["Key"]
            local_file_path = os.path.join(local_folder_path, os.path.basename(data_key))
            s3_client.download_file(bucket, data_key, local_file_path)
            print(f"Downloaded {data_location}{data_key} to {local_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

Downloaded s3://project-4-files/csv/csv/calc_case_description_test_set.csv to /home/ec2-user/SageMaker/csv/calc_case_description_test_set.csv
Downloaded s3://project-4-files/csv/csv/calc_case_description_train_set.csv to /home/ec2-user/SageMaker/csv/calc_case_description_train_set.csv
Downloaded s3://project-4-files/csv/csv/dicom_info.csv to /home/ec2-user/SageMaker/csv/dicom_info.csv
Downloaded s3://project-4-files/csv/csv/mass_case_description_test_set.csv to /home/ec2-user/SageMaker/csv/mass_case_description_test_set.csv
Downloaded s3://project-4-files/csv/csv/mass_case_description_train_set.csv to /home/ec2-user/SageMaker/csv/mass_case_description_train_set.csv
Downloaded s3://project-4-files/csv/csv/meta.csv to /home/ec2-user/SageMaker/csv/meta.csv


In [105]:
# exract the jpeg folders

local_folder_path = "/home/ec2-user/SageMaker/jpeg"
os.makedirs(local_folder_path, exist_ok=True)

data_locations = [
    "s3://project-4-files/jpeg/"
]

for data_location in data_locations:
    s3_uri_parts = data_location.split("/")
    bucket = s3_uri_parts[2]
    
    try:
        for obj in s3_client.list_objects(Bucket=bucket, Prefix="jpeg/")["Contents"]:
            data_key = obj["Key"]
            local_file_path = os.path.join(local_folder_path, os.path.basename(data_key))
            s3_client.download_file(bucket, data_key, local_file_path)
            print(f"Downloaded {data_location}{data_key} to {local_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


Downloaded s3://project-4-files/jpeg/jpeg/1.3.6.1.4.1.9590.100.1.2.100018879311824535125115145152454291132/1-263.jpg to /home/ec2-user/SageMaker/jpeg/1-263.jpg
Downloaded s3://project-4-files/jpeg/jpeg/1.3.6.1.4.1.9590.100.1.2.100018879311824535125115145152454291132/2-241.jpg to /home/ec2-user/SageMaker/jpeg/2-241.jpg
Downloaded s3://project-4-files/jpeg/jpeg/1.3.6.1.4.1.9590.100.1.2.100131208110604806117271735422083351547/1-126.jpg to /home/ec2-user/SageMaker/jpeg/1-126.jpg
Downloaded s3://project-4-files/jpeg/jpeg/1.3.6.1.4.1.9590.100.1.2.100522099512256189513864912954167862869/1-231.jpg to /home/ec2-user/SageMaker/jpeg/1-231.jpg
Downloaded s3://project-4-files/jpeg/jpeg/1.3.6.1.4.1.9590.100.1.2.100522676511025180541602449080267145647/1-111.jpg to /home/ec2-user/SageMaker/jpeg/1-111.jpg
Downloaded s3://project-4-files/jpeg/jpeg/1.3.6.1.4.1.9590.100.1.2.10055294210766234140934516480682841962/1-031.jpg to /home/ec2-user/SageMaker/jpeg/1-031.jpg
Downloaded s3://project-4-files/jpeg/jpeg

KeyboardInterrupt: 